### Variáveis de Decisão
Para cada enfermeiro $i$ e dia $j$:
* $d_{ij}$: 1 se o enfermeiro trabalha no turno **Diurno** (12h), 0 caso contrário.
* $n_{ij}$: 1 se o enfermeiro trabalha no turno **Noturno** (12h), 0 caso contrário.
* $dn_{ij}$: 1 se o enfermeiro trabalha no turno **24 Horas**, 0 caso contrário.


### Função Objetivo
Minimizar o número de solicitações de folgas não atendidas.
$$\text{Min } Z = \sum_{i=1}^{n}\sum_{j=1}^{d} p_{ij} \cdot (d_{ij} + n_{ij} + dn_{ij})$$
Onde $p_{ij} = 1$ se o enfermeiro $i$ pediu folga no dia $j$, e 0 caso contrário.

### Ferramentas Utilizadas

Para a implementação computacional do modelo, optou-se pela utilização da linguagem Python em conjunto com a biblioteca PuLP. Essa escolha justifica-se pela sua natureza open-source, pela facilidade de integração com estruturas de dados modernas e pela sua sintaxe clara, que permite transcrever as restrições matemáticas de forma direta e legível.

Para ver a documentação, acesse o site oficial da [Biblioteca PuLP](https://coin-or.github.io/pulp/).

### 1. PARÂMETROS DO MODELO

In [ ]:
import pulp
import random

num_enfermeiros = 13  
dias_no_mes = 28      
enfermeiros = [f'Enf_{i+1}' for i in range(num_enfermeiros)]
dias = range(1, dias_no_mes + 1)
turnos = ['D', 'N', 'DN'] 

C_min_D = 2   
C_max_D = 5   
C_min_N = 2  
C_max_N = 5

H_max = 44 
P_min = 10   

semanas = [
    range(1, 8),   
    range(8, 15),  
    range(15, 22), 
    range(22, 29)  
]

random.seed(42) 
pedidos_folga = {}
for e in enfermeiros:
    for d in dias:
        pedidos_folga[(e, d)] = 1 if random.random() > 0.5 else 0

print("Dados carregados com sucesso.")

Dados carregados com sucesso.


### 2. CONSTRUÇÃO DO MODELO

In [ ]:
prob = pulp.LpProblem("Escala_Enfermeiros", pulp.LpMinimize)

x = pulp.LpVariable.dicts("Escala", 
                          ((e, d, t) for e in enfermeiros for d in dias for t in turnos),
                          cat='Binary')

prob += pulp.lpSum([
    pedidos_folga[(e,d)] * (x[(e,d,'D')] + x[(e,d,'N')] + x[(e,d,'DN')])
    for e in enfermeiros
    for d in dias
]), "Minimizar_Insatisfacao"

for d in dias:
    total_diurno = pulp.lpSum([x[(e,d,'D')] + x[(e,d,'DN')] for e in enfermeiros])
    prob += total_diurno >= C_min_D
    prob += total_diurno <= C_max_D
    
    total_noturno = pulp.lpSum([x[(e,d,'N')] + x[(e,d,'DN')] for e in enfermeiros])
    prob += total_noturno >= C_min_N
    prob += total_noturno <= C_max_N

for e in enfermeiros:
    
    for d in dias:
        prob += x[(e,d,'D')] + x[(e,d,'N')] + x[(e,d,'DN')] <= 1

    prob += pulp.lpSum([
        x[(e,d,'D')] + x[(e,d,'N')] + 2 * x[(e,d,'DN')] 
        for d in dias
    ]) >= P_min
    
    for semana in semanas:
        dias_semana = [d for d in semana if d in dias]
        horas_semana = pulp.lpSum([
            12 * x[(e,d,'D')] + 12 * x[(e,d,'N')] + 24 * x[(e,d,'DN')]
            for d in dias_semana
        ])
        prob += horas_semana <= H_max

print("Modelo construido.")

Modelo construido.


### 3. RESOLUÇÃO E VISUALIZAÇÃO

In [ ]:
status = prob.solve()
print(f"Status da Solucao: {pulp.LpStatus[status]}")

if status == pulp.LpStatusOptimal:
    print(f"Total de pedidos de folga nao atendidos (Funcao Objetivo): {pulp.value(prob.objective)}")
    print("-" * 100)
    
    header = f"{'Enf.':<8} | " + " ".join([f"{d:02d}" for d in dias])
    print(header)
    print("-" * 100)
    
    for e in enfermeiros:
        linha = f"{e:<8} | "
        for d in dias:
            cell = "."
            if pulp.value(x[(e,d,'D')]) == 1: cell = "D "
            if pulp.value(x[(e,d,'N')]) == 1: cell = "N "
            if pulp.value(x[(e,d,'DN')]) == 1: cell = "DN"
            
            if pedidos_folga[(e,d)] == 1 and cell != ".":
                cell = cell.strip() + "*"
                cell = f"{cell:<2}" 
            
            linha += cell + " "
        print(linha)
        
    print("-" * 100)
    print("Legenda: D=Diurno, N=Noturno, DN=24h, * = Pedido de folga ignorado")

else:
    print("Nao foi possivel encontrar uma solucao otima.")

Status da Solucao: Optimal
Total de pedidos de folga nao atendidos (Funcao Objetivo): 1.0
----------------------------------------------------------------------------------------------------
Enf.     | 01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28
----------------------------------------------------------------------------------------------------
Enf_1    | DN . . . . . D  . . . . DN . . N  . . . D  . D  D  . DN* . . . . 
Enf_2    | . . . N  DN . . DN . . D  . . . . . . . . . N  . . . . . DN N  
Enf_3    | . . . . N  . N  N  . DN . . . . . . D  . N  . . . . . D  . DN . 
Enf_4    | . . . DN . . . . DN . . . . . D  . . . DN . . . . . . N  DN . 
Enf_5    | . . D  . . D  D  . DN . . D  . . . N  N  . . . . N  . . . N  . . 
Enf_6    | . DN . . . . N  . . . N  . . DN . . . N  . . . N  . DN . . . . 
Enf_7    | . . N  . . N  D  . . . . . N  DN DN . . . . . . . . . . DN . . 
Enf_8    | . . N  . D  D  . . . . . . . DN . N  . . . DN . . . . . D  . D  
Enf_9   

In [ ]:
total_pedidos = 0
total_nao_atendidos = 0

for e in enfermeiros:
    for d in dias:
        if pedidos_folga.get((e, d), 0) == 1:
            total_pedidos += 1
            
            trabalhou = (pulp.value(x[(e,d,'D')]) == 1 or 
                         pulp.value(x[(e,d,'N')]) == 1 or 
                         pulp.value(x[(e,d,'DN')]) == 1)
            
            if trabalhou:
                total_nao_atendidos += 1

total_atendidos = total_pedidos - total_nao_atendidos
taxa_sucesso = (total_atendidos / total_pedidos * 100) if total_pedidos > 0 else 100

print("ESTATÍSTICAS DE SATISFAÇÃO DA EQUIPE")
print()
print(f"Total de solicitações de folga: {total_pedidos}")
print(f"Solicitações atendidas:         {total_atendidos}")
print(f"Solicitações NÃO atendidas:     {total_nao_atendidos}")
print(f"Taxa de atendimento:            {taxa_sucesso:.2f}%")

ESTATÍSTICAS DE SATISFAÇÃO DA EQUIPE

Total de solicitações de folga: 178
Solicitações atendidas:         177
Solicitações NÃO atendidas:     1
Taxa de atendimento:            99.44%


In [5]:
import pandas as pd

if status == pulp.LpStatusOptimal:
    dados_escala = {}

    for e in enfermeiros:
        linha_enfermeiro = []
        for d in dias:
            turno_dia = "Folga"
            
            if pulp.value(x[(e,d,'D')]) == 1:
                turno_dia = "D"
            elif pulp.value(x[(e,d,'N')]) == 1:
                turno_dia = "N"
            elif pulp.value(x[(e,d,'DN')]) == 1:
                turno_dia = "DN"
            
            if pedidos_folga.get((e,d), 0) == 1 and turno_dia != "Folga":
                turno_dia += "(Folga nao atendida)"
            
            linha_enfermeiro.append(turno_dia)
        
        dados_escala[e] = linha_enfermeiro

    df_escala = pd.DataFrame.from_dict(dados_escala, orient='index', columns=[f'Dia {d}' for d in dias])

    display(df_escala.T)
else:
    print("O problema nao tem solucao otima.")

,Enf_1,Enf_2,Enf_3,Enf_4,Enf_5,Enf_6,Enf_7,Enf_8,Enf_9,Enf_10,Enf_11,Enf_12,Enf_13
Dia 1,DN,Folga,Folga,Folga,Folga,Folga,Folga,Folga,Folga,Folga,DN,Folga,Folga
Dia 2,Folga,Folga,Folga,Folga,Folga,DN,Folga,Folga,N,Folga,Folga,D,D
Dia 3,Folga,Folga,Folga,Folga,D,Folga,N,N,Folga,Folga,Folga,D,Folga
Dia 4,Folga,N,Folga,DN,Folga,Folga,Folga,Folga,Folga,DN,Folga,Folga,Folga
Dia 5,Folga,DN,N,Folga,Folga,Folga,Folga,D,Folga,Folga,Folga,Folga,Folga
Dia 6,Folga,Folga,Folga,Folga,D,Folga,N,D,DN,Folga,Folga,Folga,Folga
Dia 7,D,Folga,N,Folga,D,N,D,Folga,Folga,Folga,N,D,Folga
Dia 8,Folga,DN,N,Folga,Folga,Folga,Folga,Folga,Folga,D,Folga,Folga,Folga
Dia 9,Folga,Folga,Folga,DN,DN,Folga,Folga,Folga,Folga,Folga,Folga,Folga,Folga
Dia 10,Folga,Folga,DN,Folga,Folga,Folga,Folga,Folga,D,Folga,Folga,Folga,N
